In [1]:
from pyCode.AuxDecays import DecayList,Decay
from pyCode.component import Component
from pyCode.AuxFuncs import T as Tfunc

INFO:pyCode.AuxFuncs:Loading aux functions. Ignoring BSM corrections to g* and g*_S


In [2]:
import numpy as np
import sympy as sp

In [3]:
lamb = 0.17
decays = DecayList()
decayToDM = Decay(instate='Mediator',fstates=['DM','radiation'],br=1.)
decays.addDecay(decayToDM)
decays.Xfraction = 0.1
decays.width = 2.49e-15*(lamb/4.3e-7)**2
dofDM = -2
dofMed = 6
dm = Component(label='DM',Type='thermal',dof=dofDM, mass=500.)
mediator = Component(label='Mediator',Type='thermal',dof=dofMed, mass=510.,decays=decays)
components = [dm,mediator]

In [4]:
#Store the number of components:
nComp = len(components)

n = np.array(sp.symbols('n:%d'%nComp))
Ri = np.array(sp.symbols('R:%d'%nComp))
NS = sp.symbols('NS')
x = sp.symbols('x')
T = Tfunc(x,NS)

#Current energy densities:
rho = n*Ri

#Compute equilibrium densities:
neq = np.array([comp.nEQ(T) for comp in components])

#Compute ratio of equilibrium densities
#(helps with numerical instabilities)
#rNeq[i,j] = neq[i]/neq[j]
rNeq = np.array([[compi.rNeq(T,compj) if compi.active and compj.active else 0. for compj in components] 
                 for compi in components])

#Dictionary with label:index mapping:
labelsDict = dict([[comp.label,i] for i,comp in enumerate(components)])

#Compute Hubble factor:
# H = Hfunc(T,rho,isActive)

#Effective equilibrium densities and BRs:
NXth = np.array([comp.getNXTh(T,n,rNeq,labelsDict) for comp in components])
widths = np.array([comp.width(T) for comp in components])
masses = np.array([comp.mass(T) for comp in components])
BRX = np.array([comp.getBRX(T) for comp in components])
sigmaV = np.array([comp.getSIGV(T) for comp in components])

In [5]:
RHS = -3*n
#Decay term:
RHS -= widths*masses*n/(Ri)
#Inverse decay term:
RHS += widths*masses*NXth/(Ri) #NXth should be finite if i -> j +..
#Annihilation term:            
RHS += sigmaV*(neq - n)*(neq + n)

In [7]:
RHS[0]

-3*n0

In [8]:
fff = sp.lambdify([x,n,Ri,NS],RHS, modules=[{'DiracDelta' : lambda z: 0.},'numpy','sympy'])

In [9]:
fff(1.,np.array([10.,1.]),np.array([600.,600.]),2.)

[-30.0, nan]

In [10]:
ff = sp.lambdify([x,NS],Tfunc(x,NS))

In [11]:
ff(1.,2.)

0.25406166871224

In [9]:
vv = np.hstack((n,Ri,[T]))
jac = sp.Matrix(RHS).jacobian(vv)
jacf = sp.lambdify(vv,jac,modules=[{'DiracDelta' : lambda x: 0.},'numpy','sympy'])

ValueError: 
Can't calculate derivative wrt T(x, NS).

In [ ]:
jacf(10.,1.,600.,600.,1000.)